In [ ]:
#wan
model_train_list = {
  "clip_l": "https://huggingface.co/StableDiffusionVN/Flux/blob/main/Clip/clip_l.safetensors",
  "t5_xxl": "https://huggingface.co/StableDiffusionVN/Flux/blob/main/Clip/t5xxl_fp16.safetensors",
  "flux_vae": "https://huggingface.co/StableDiffusionVN/Flux/blob/main/Vae/flux_vae.safetensors",

  "kontext_dev": "https://huggingface.co/StableDiffusionVN/Flux/blob/main/Unet/flux1-kontext-dev.safetensors",
  "qwen_image_bf16": "https://huggingface.co/Comfy-Org/Qwen-Image_ComfyUI/blob/main/split_files/diffusion_models/qwen_image_bf16.safetensors",
  "qwen_image_edit_bf16": "https://huggingface.co/Comfy-Org/Qwen-Image-Edit_ComfyUI/blob/main/split_files/diffusion_models/qwen_image_edit_bf16.safetensors",
  "qwen_image_edit_2509_bf16": "https://huggingface.co/Comfy-Org/Qwen-Image-Edit_ComfyUI/blob/main/split_files/diffusion_models/qwen_image_edit_2509_bf16.safetensors",
  "qwen_image_edit_2511_bf16": "https://huggingface.co/Comfy-Org/Qwen-Image-Edit_ComfyUI/blob/main/split_files/diffusion_models/qwen_image_edit_2511_bf16.safetensors",
  "clip_qwen": "https://huggingface.co/Comfy-Org/Qwen-Image_ComfyUI/blob/main/split_files/text_encoders/qwen_2.5_vl_7b.safetensors",
  "vae_qwen": "https://huggingface.co/StableDiffusionVN/QwenImage/blob/main/vae/qwen_vae.safetensors",

  "z_image_turbo_bf16": "https://huggingface.co/Comfy-Org/z_image_turbo/blob/main/split_files/diffusion_models/z_image_turbo_bf16.safetensors",
  "z_image_de_turbo_v1_bf16": "https://huggingface.co/ostris/Z-Image-De-Turbo/blob/main/z_image_de_turbo_v1_bf16.safetensors",
  "qwen_3_4b": "https://huggingface.co/Comfy-Org/z_image_turbo/blob/main/split_files/text_encoders/qwen_3_4b.safetensors",
  "vae_z": "https://huggingface.co/Comfy-Org/z_image_turbo/blob/main/split_files/vae/ae.safetensors",
  "z_adapter_v2": "https://huggingface.co/ostris/zimage_turbo_training_adapter/blob/main/zimage_turbo_training_adapter_v2.safetensors",
  
  "wan21_t2v_14B": "https://huggingface.co/Comfy-Org/Wan_2.1_ComfyUI_repackaged/blob/main/split_files/diffusion_models/wan2.1_t2v_14B_fp8_e4m3fn.safetensors",
  "wan21_i2v_14B": "https://huggingface.co/Comfy-Org/Wan_2.1_ComfyUI_repackaged/blob/main/split_files/diffusion_models/wan2.1_i2v_720p_14B_fp8_e4m3fn.safetensors",

  "vae_wan21": "https://huggingface.co/Comfy-Org/Wan_2.1_ComfyUI_repackaged/blob/main/split_files/vae/wan_2.1_vae.safetensors",
  "umt5_clip": "https://huggingface.co/Wan-AI/Wan2.1-T2V-1.3B/blob/main/models_t5_umt5-xxl-enc-bf16.pth",
  "wan_clip_vision": "https://huggingface.co/alibaba-pai/Wan2.1-Fun-14B-Control/blob/main/models_clip_open-clip-xlm-roberta-large-vit-huge-14.pth",

  "wan22_t2v_low_14B": "https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/blob/main/split_files/diffusion_models/wan2.2_t2v_low_noise_14B_fp16.safetensors",
  "wan22_t2v_high_14B": "https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/blob/main/split_files/diffusion_models/wan2.2_t2v_high_noise_14B_fp16.safetensors",

  "wan22_i2v_low_14B": "https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/blob/main/split_files/diffusion_models/wan2.2_i2v_low_noise_14B_fp16.safetensors",
  "wan22_i2v_high_14B": "https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/blob/main/split_files/diffusion_models/wan2.2_i2v_high_noise_14B_fp16.safetensors",
}

#model_dic

model_dic = {
    "kontext_dev": {
        "model": "kontext_dev",
        "vae": "flux_vae",
        "clip": "t5_xxl",
        "clip2": "clip_l",
    },
    "qwen_image": {
        "model": "qwen_image_bf16",
        "vae": "vae_qwen",
        "clip": "clip_qwen",
    },
    "qwen_image_edit": {
        "model": "qwen_image_edit_bf16",
        "vae": "vae_qwen",
        "clip": "clip_qwen",
    },
    "qwen_image_edit_2509": {
        "model": "qwen_image_edit_2509_bf16",
        "vae": "vae_qwen",
        "clip": "clip_qwen",
    },
    "qwen_image_edit_2511": {
        "model": "qwen_image_edit_2511_bf16",
        "vae": "vae_qwen",
        "clip": "clip_qwen",
    },
    "z_image_turbo": {
        "model": "z_image_de_turbo_v1_bf16",
        "vae": "vae_z",
        "clip": "qwen_3_4b",
        "adaptar": "z_adapter_v2"
    },
    "finetuning-z_image_turbo": {
        "model": "z_image_de_turbo_v1_bf16",
        "vae": "vae_z",
        "clip": "qwen_3_4b",
    },
    "finetuning-qwen_image": {
        "model": "qwen_image_bf16",
        "vae": "vae_qwen",
        "clip": "clip_qwen",
    },
    "finetuning-qwen_image_edit": {
        "model": "qwen_image_edit_bf16",
        "vae": "vae_qwen",
        "clip": "clip_qwen",
    },
    "finetuning-qwen_image_edit_2509": {
        "model": "qwen_image_edit_2509_bf16",
        "vae": "vae_qwen",
        "clip": "clip_qwen",
    },
    "finetuning-qwen_image_edit_2511": {
        "model": "qwen_image_edit_2511_bf16",
        "vae": "vae_qwen",
        "clip": "clip_qwen",
    },
    "wan21_t2v_14B": {
        "model": "wan21_t2v_14B",
        "vae": "vae_wan21",
        "clip": "umt5_clip",
        "task": "t2v-14B",
        "discrete_flow_shift": 3,
    },
    "wan21_i2v_14B": {
        "model": "wan21_i2v_14B",
        "vae": "vae_wan21",
        "clip": "umt5_clip",
        "clip_vision":"wan_clip_vision",
        "task": "i2v-14B",
        "discrete_flow_shift": 3,
    },
    "wan22_t2v_14B": {
        "model": "wan22_t2v_low_14B",
        "model2": "wan22_t2v_high_14B",
        "vae": "vae_wan21",
        "clip": "umt5_clip",
        "task": "t2v-A14B",
        "discrete_flow_shift": 12,
    },
    "wan22_i2v_14B": {
        "model": "wan22_i2v_low_14B",
        "model2": "wan22_i2v_high_14B",
        "vae": "vae_wan21",
        "clip": "umt5_clip",
        "task": "i2v-A14B",
        "discrete_flow_shift": 5,
    },
}

model_folder = os.path.join(root_dir,"models")

In [ ]:
def dic2arg(config:dict):
  arg = ''
  for value in config:
    arg += f'{value if str(config[value]) != "False" else ""} {"" if type(config[value]) == bool else config[value]} '
  return arg

def hug_down(link,path):
  name = path.split('/')[-1]
  folder = path.split(name)[0]
  if "blob" in link:
    link = link.replace("blob","resolve")
  !aria2c --console-log-level=error --summary-interval=10 -c -x 16 -s 16 -k 1M {link} -d {folder} -o {name}

def model_path(model):
  model_name = model
  model_name = model_train_list[model].split('/')[-1]
  model_path = os.path.join(model_folder,model_name)
  if not os.path.exists(model_path):
    hug_down(model_train_list[model],model_path)
  return model_path

op = {
    '--mixed_precision': 'bf16',
    '--num_cpu_threads_per_process': 1,
}

qwen_version = {
    "qwen_image": "original",
    "qwen_image_edit": "edit",
    "qwen_image_edit_2509": "edit-2509",
    "qwen_image_edit_2511": "edit-2511"
}

if "kontext" in model_type:
  timestep_sampling = "flux_shift"
  networks = "networks.lora_flux"
elif "qwen" in model_type:
  discrete_flow_shift = 2.2
  timestep_sampling = "shift"
  networks = "networks.lora_qwen_image"
  
elif "z_image" in model_type:
  discrete_flow_shift = 2.0
  timestep_sampling = "shift"
  networks = "networks.lora_zimage"

def fun_data_config():
  return {
    "general": {
        "resolution": resolution,
        "caption_extension": extension,
        "batch_size": batch_size,
        "enable_bucket": True,
        "bucket_no_upscale": False
    },
    "datasets": []
  }

def fun_config():
  dataset_config_path = os.path.join(root_dir,"dataset.toml")
  return {
    "flux_kontext_train_network.py": True if "kontext" in model_type else False,
    "qwen_image_train_network.py": True if "qwen" in model_type else False,
    "zimage_train_network.py": True if "z_image" in model_type else False,
    "--dit": f'"{model_path(model_dic[model_type]["model"])}"' if model_train == "" else download_lib(model_train),
    "--vae": f'"{model_path(model_dic[model_type]["vae"])}"',
    "--text_encoder": f'"{model_path(model_dic[model_type]["clip"])}"' if "clip2" not in model_dic[model_type] else False,
    "--text_encoder1": f'"{model_path(model_dic[model_type]["clip"])}"' if "clip2" in model_dic[model_type] else False,
    "--text_encoder2": f'"{model_path(model_dic[model_type]["clip2"])}"' if "clip2" in model_dic[model_type] else False,
    "--dataset_config": f'"{dataset_config_path}"',
    "--model_version": qwen_version[model_type] if "qwen" in model_type else False,
    "--sdpa": True,
    "--flash_attn": False,
    "--flash3": False,
    "--sage_attn": False,
    "--xformers": False,
    "--split_attn": True,
    "--mixed_precision": "bf16",
    "--fp8_base": fp8_base,
    "--fp8_vl": fp8_vl if "qwen" in model_type else False,
    "--optimizer_type": optimizer_type,
    "--optimizer_args": False,
    "--learning_rate": learning_rate,
    "--lr_scheduler": lr_scheduler,
    "--lr_warmup_steps": lr_warmup_steps if lr_scheduler == "constant_with_warmup" else False,
    "--lr_scheduler_power": lr_poly_power if lr_scheduler == "polynomial" else False,
    "--lr_scheduler_num_cycles": lr_restarts_num_cycles if lr_scheduler == "cosine_with_restarts" else False,
    "--gradient_checkpointing": gradient_checkpointing,
    "--gradient_accumulation_steps": False,
    "--max_data_loader_n_workers": 2,
    "--persistent_data_loader_workers": True,
    "--network_module": networks,
    "--network_dim": network_dim,
    "--network_alpha": network_alpha,
    "--network_dropout": False,
    "--network_weights": False,
    "--blocks_to_swap": 16 if blocks_to_swap and "qwen" in model_type else False,
    "--timestep_sampling": timestep_sampling,
    "--min_timestep": min_timestep,
    "--max_timestep": max_timestep,
    "--preserve_distribution_shape": preserve_distribution_shape,
    "--discrete_flow_shift": discrete_flow_shift,
    "--weighting_scheme": "none",
    "--max_train_steps": False if max_train_steps <= 0 else max_train_steps,
    "--max_train_epochs": False if max_train_epochs <= 0 else max_train_epochs,
    "--save_every_n_epochs": 1 if save_every_n_epochs < 1 else save_every_n_epochs,
    "--save_last_n_epochs": False if save_last_n_epochs <= 0 else save_last_n_epochs,
    "--save_every_n_steps": False if save_every_n_steps <= 0 else save_every_n_steps,
    "--seed": 42,
    "--output_dir": f'"{output_dir}/{output_name}_{model_type}"',
    "--output_name": f'"{output_name}"'
}

def fun_config_finetuning():
  dataset_config_path = os.path.join(root_dir,"dataset.toml")
  return {
    "zimage_train.py": True if "z_image" in model_type else False,
    "qwen_image_train.py": True if "qwen" in model_type else False,
    "--dit": f'"{model_path(model_dic[model_type]["model"])}"' if model_train == "" else download_lib(model_train),
    "--vae": f'"{model_path(model_dic[model_type]["vae"])}"',
    "--text_encoder": f'"{model_path(model_dic[model_type]["clip"])}"' if "clip2" not in model_dic[model_type] else False,
    "--text_encoder1": f'"{model_path(model_dic[model_type]["clip"])}"' if "clip2" in model_dic[model_type] else False,
    "--text_encoder2": f'"{model_path(model_dic[model_type]["clip2"])}"' if "clip2" in model_dic[model_type] else False,
    "--dataset_config": f'"{dataset_config_path}"',
    "--model_version": qwen_version[model_type] if "qwen" in model_type else False,
    "--sdpa": True,
    "--split_attn": True,
    "--mixed_precision": "bf16",
    "--full_bf16": full_bf16,
    "--gradient_checkpointing": gradient_checkpointing,
    "--optimizer_type": "adafactor",
    "--fused_backward_pass": True,
    "--optimizer_args": '"relative_step=False" "scale_parameter=False" "warmup_init=False"',
    "--learning_rate": learning_rate,
    "--lr_scheduler": lr_scheduler,
    "--lr_warmup_steps": lr_warmup_steps if lr_scheduler == "constant_with_warmup" else False,
    "--lr_scheduler_power": lr_poly_power if lr_scheduler == "polynomial" else False,
    "--lr_scheduler_num_cycles": lr_restarts_num_cycles if lr_scheduler == "cosine_with_restarts" else False,
    "--timestep_sampling": timestep_sampling,
    "--min_timestep": min_timestep,
    "--max_timestep": max_timestep,
    "--preserve_distribution_shape": preserve_distribution_shape,
    "--max_data_loader_n_workers": 2,
    "--persistent_data_loader_workers": True,
    "--max_train_steps": False if max_train_steps <= 0 else max_train_steps,
    "--max_train_epochs": False if max_train_epochs <= 0 else max_train_epochs,
    "--save_every_n_epochs": 1 if save_every_n_epochs < 1 else save_every_n_epochs,
    "--save_last_n_epochs": False if save_last_n_epochs <= 0 else save_last_n_epochs,
    "--save_every_n_steps": False if save_every_n_steps <= 0 else save_every_n_steps,
    "--seed": 42,
    "--output_dir": f'"{output_dir.split("/Lora")[0]}/ComfyModel/diffusion_models/{output_name}_{model_type}"',
    "--output_name": f'"{output_name}"'
}



dataset_config_path = os.path.join(root_dir,"dataset.toml")
if "bf16" in model_train_list[model_dic[model_type]["model"]]:
  mixed_precision = "bf16"
elif "fp16" in model_train_list[model_dic[model_type]["model"]]:
  mixed_precision = "fp16"
else:
  mixed_precision = None

def fun_config_video():
  dataset_config_path = os.path.join(root_dir,"dataset.toml")
  return {
    "wan_train_network.py": True if "wan" in model_type else False,
    "--task": model_dic[model_type]["task"] if "wan" in model_type else False,
    "--dit": f'"{model_path(model_dic[model_type]["model"])}"',
    "--dit_high_noise": f'"{model_path(model_dic[model_type]["model2"])}"' if "model2" in model_dic[model_type] else False,
    "--vae": f'"{model_path(model_dic[model_type]["vae"])}"',
    "--t5": f'"{model_path(model_dic[model_type]["clip"])}"' if "wan" in model_type else False,
    "--clip": f'"{model_path(model_dic[model_type]["clip_vision"])}"' if "wan21_i2v" in model_type else False,
    "--dataset_config": f'"{dataset_config_path}"',
    "--sdpa": True,
    "--split_attn": True,
    "--mixed_precision": mixed_precision,
    "--fp8_base": fp8_base,
    "--optimizer_type": optimizer_type,
    "--optimizer_args": False,
    "--learning_rate": learning_rate,
    "--lr_scheduler": lr_scheduler,
    "--lr_warmup_steps": lr_warmup_steps if lr_scheduler == "constant_with_warmup" else False,
    "--lr_scheduler_power": lr_poly_power if lr_scheduler == "polynomial" else False,
    "--lr_scheduler_num_cycles": lr_restarts_num_cycles if lr_scheduler == "cosine_with_restarts" else False,
    "--gradient_checkpointing": gradient_checkpointing,
    "--gradient_accumulation_steps": False,
    "--max_data_loader_n_workers": 2,
    "--persistent_data_loader_workers": True,
    "--network_module": "networks.lora_wan",
    "--network_dim": network_dim,
    "--network_alpha": network_alpha,
    "--network_dropout": False,
    "--network_weights": False,
    "--timestep_sampling": timestep_sampling,
    "--min_timestep": min_timestep,
    "--max_timestep": max_timestep,
    "--preserve_distribution_shape": preserve_distribution_shape,
    "--discrete_flow_shift": model_dic[model_type]["discrete_flow_shift"],
    "--timestep_boundary": timestep_boundary if "wan22" in model_type else False,
    "--max_train_steps": max_train_steps if max_train_steps > 0 else False,
    "--max_train_epochs": max_train_epochs if max_train_epochs > 0 else False,
    "--save_every_n_epochs": save_every_n_epochs if save_every_n_epochs >= 1 else 1,
    "--save_last_n_epochs": False if save_last_n_epochs <= 0 else save_last_n_epochs,
    "--save_every_n_steps": False if save_every_n_steps <= 0 else save_every_n_steps,
    "--seed": 42,
    "--output_dir": f'"{output_dir}/{output_name}_{model_type}"',
    "--output_name": f'"{output_name}"'
}

sample_prompt_path = os.path.join(root_dir,"prompt.txt")

def fun_extra():
  return {
    "--sample_prompts": f'"{sample_prompt_path}"' if sample_every_n_steps > 0 else False,
    "--sample_every_n_epochs": False,
    "--sample_every_n_steps": sample_every_n_steps if sample_every_n_steps > 0 else False,
    "--sample_at_first": True if sample_every_n_steps > 0 else False,
    "--log_with": "wandb" if wandb_api_key != "" else False,
    "--wandb_api_key": wandb_api_key if wandb_api_key != "" else False,
    "--wandb_run_name": f'"{model_type}_{output_name}"',
    "--save_state": save_state,
    "--save_state_on_train_end": save_state_on_train_end,
    "--resume": resume if resume != "" else False,
    '--metadata_title': f'"{output_name}"',
    '--metadata_author': f'"{author}"',
    '--metadata_description': f'"{description}"',
    '--metadata_license': False, # "MIT"
    '--metadata_tags': False, # "sdvn.me"
}

def fun_prompt(sample_prompt, sample_image_path):
  if "kontext" in model_type or "edit" in model_type:
    qwen_size = True if 'qwen' in model_type else False
    if ControlFolder != "":
      s, p = random_sample(folder_train,ControlFolder)
    else:
      s, p = random_sample(folder_train,folder_train)
    if sample_prompt == "" :
      sample_prompt = s
    if sample_image_path == "":
      sample_image_path = p
    prompt = f"""{sample_prompt} --w {image_size(sample_image_path, qwensize = qwen_size)[0]} --h {image_size(sample_image_path, qwensize = qwen_size)[1]} --f 25 --ci {sample_image_path}"""
  else:
    prompt = f"""{sample_prompt if sample_prompt != "" else random_sample(folder_train)[0]} --w {sample_size[0]} --h {sample_size[1]} --f 25"""

  write_file(sample_prompt_path,prompt)

def fun_prompt_video(sample_prompt, sample_image_path):
  if "i2v" in model_type:
    prompt = f"""{sample_prompt if sample_prompt != "" else random_sample(folder_train)} --w {sample_size[0]} --h {sample_size[1]} --f 25 --i {sample_image_path}"""
  else:
    prompt = f"""{sample_prompt if sample_prompt != "" else random_sample(folder_train)} --w {sample_size[0]} --h {sample_size[1]} --f 25"""

  write_file(sample_prompt_path,prompt)


def fun_dataset_file():
  dataset = os.path.join(root_dir,"dataset.toml")
  for dir in check_sub_dir(image_dir):
    data_config["datasets"].append({
        "image_directory": dir,
        "control_directory": dir.replace(ImageFolder, ControlFolder) if ControlFolder != "" else None,
        "cache_directory": os.path.join(root_dir,f"cache_{model_type}{dir.split(image_dir)[-1]}"),
        "num_repeats": repeat_dir(dir,num_repeats),
        "flux_kontext_no_resize_control": True if resize_control == False and "kontext" in model_type else None,
        })

  with open(dataset, "w") as file:
      toml.dump(data_config, file)

def fun_dataset_file_video():
  dataset = os.path.join(root_dir,"dataset.toml")
  if Image_train:
    for dir in check_sub_dir(image_dir):
      data_config["datasets"].append({
            "image_directory": dir,
            "cache_directory": os.path.join(root_dir,f"cache_{model_type}"),
            "num_repeats": repeat_dir(dir,num_repeats),
            })
  if Video_train:
    for dir in check_sub_dir(video_dir):
      data_config["datasets"].append({
            "video_directory": dir,
            "cache_directory": os.path.join(root_dir,f"cachevideo_{model_type}"),
            "num_repeats": repeat_dir(dir,num_repeats),
            "frame_extraction": frame_extraction,
            "target_frames": target_frames,
            "frame_stride": frame_stride,
            "frame_sample": frame_sample,
            "max_frames": max_frames,
        })

  with open(dataset, "w") as file:
      toml.dump(data_config, file)

dataset_config_path = os.path.join(root_dir,"dataset.toml")

def fun_cache_latents_config():
  return {
    "python": True,
    "flux_kontext_cache_latents.py": True if "kontext" in model_type else False,
    "src/musubi_tuner/qwen_image_cache_latents.py": True if "qwen" in model_type else False,
    "src/musubi_tuner/zimage_cache_latents.py": True if "z_image" in model_type else False,
    "--dataset_config": f'"{dataset_config_path}"',
    "--vae": f'"{model_path(model_dic[model_type]["vae"])}"',
    "--model_version": qwen_version[model_type] if "qwen" in model_type else False,
}

def fun_cache_latents_config_video():
  return {
    "python": True,
    "wan_cache_latents.py": True if "wan" in model_type else False,
    "--dataset_config": f'"{dataset_config_path}"',
    "--vae": f'"{model_path(model_dic[model_type]["vae"])}"',
    "--i2v": True if "i2v" in model_type else False,
    "--clip": f'"{model_path(model_dic[model_type]["clip_vision"])}"' if "wan21_i2v" in model_type else False,
}


def fun_cache_text_encoder_config():
  return {
    "python": True,
    "flux_kontext_cache_text_encoder_outputs.py": True if "kontext" in model_type else False,
    "src/musubi_tuner/qwen_image_cache_text_encoder_outputs.py": True if "qwen" in model_type else False,
    "src/musubi_tuner/zimage_cache_text_encoder_outputs.py": True if "z_image" in model_type else False,
    "--dataset_config": f'"{dataset_config_path}"',
    "--text_encoder": f'"{model_path(model_dic[model_type]["clip"])}"' if "clip2" not in model_dic[model_type] else False,
    "--text_encoder1": f'"{model_path(model_dic[model_type]["clip"])}"' if "clip2" in model_dic[model_type] else False,
    "--text_encoder2": f'"{model_path(model_dic[model_type]["clip2"])}"' if "clip2" in model_dic[model_type] else False,
    "--batch_size": 16,
    "--model_version": qwen_version[model_type] if "qwen" in model_type else False,
}

def fun_cache_text_encoder_config_video():
  return {
    "python": True,
    "wan_cache_text_encoder_outputs.py": True if "wan" in model_type else False,
    "--dataset_config": f'"{dataset_config_path}"',
    "--t5": f'"{model_path(model_dic[model_type]["clip"])}"' if "wan" in model_type else False,
    "--batch_size": 16
}